<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/ArabicPIIRedaction/blob/main/fine-tuning_SLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [1]:
import pandas as pd

### Test data
- Original Competition Data

In [ ]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

,source,target,dialect
0,الأستاذ أدولفوس ريغان زييمان، بصفتك المدير الت...,الأستاذ [MASK]، بصفتك المدير التنفيذي الرئيسي ...,EGYPT
1,مرحبًا يا هانا، ممكن لو سمحتي تتحققي من التداع...,مرحبًا يا [MASK]، ممكن لو سمحتي تتحققي من التد...,EGYPT
2,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,SAUDI
3,عزيزي ديفان، مطلوب تقديم عرض تقديمي على مستوى ...,عزيزي [MASK]، مطلوب تقديم عرض تقديمي على مستوى...,EGYPT
4,ممكن كمان نعمل جلسة عن كيفية إدارة التوتر والت...,The sentence doesn't contain any personal or s...,LEVANTINE


### Train data
- Created by Mosaab

In [2]:
from google.colab import drive
import random

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ar_dataset_translated_fixed.csv"

train = pd.read_csv(data_path)

In [5]:
train.drop(["Unnamed: 0",'lang'], axis=1, inplace=True)
print(len(train))
train.head()

209261


,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'ال...","['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', '...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"['نحن', 'نقوم', 'بمراجعة', 'القضية', 'التي', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-FIRSTNAME', ...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"['بسبب', 'اتفاقية', 'السرية', '،', 'لا', 'يمكن...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEX', '...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"['تنظيم', 'اجتماع', 'حول', '7', 'سنوات', '(', ...","['O', 'O', 'O', 'B-AGE', 'I-AGE', 'O', 'B-GEND...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [6]:
idx = random.randint(0, len(train))
idx

127602

In [ ]:
print(train.tokens[idx])
print(train.tags[idx])
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['بالنسبة', 'للطلاب', 'ذوي', 'الإعاقة', 'البصرية', '،', 'يتم', 'توفير', 'خدمات', 'خاصة', 'من', 'قبل', 'Paradigm', '.', 'يرجى', 'الإشارة', 'إلى', 'ذلك', 'في', '02', '/', '09', '/', '1904', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOB', 'I-DOB', 'I-DOB', 'I-DOB', 'I-DOB', 'O']
192
145


##### Cleaning and preprocessing

##### Convert tokens and tags columns to lists

In [9]:
import re
import ast

In [11]:
tokens_list = ast.literal_eval(train.tokens[idx])
print(tokens_list)
print(type(tokens_list), len(tokens_list))

tags_list = ast.literal_eval(train.tags[idx])
print(tags_list)
print(type(tags_list), len(tags_list))

['مرحبا', 'جينو', '،', 'لحماية', 'خصوصيتك', 'أثناء', 'المسابقات', 'الأكاديمية', '،', 'سوف', 'نتصل', 'بك', 'باسمك', 'الرمزي', 'Travon_Wintheiser', '@', 'hotmail', '.', 'com', '.', 'سيتم', 'استخدام', 'هذا', 'في', 'جميع', 'اتصالاتنا', '.']
<class 'list'> 27
['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
<class 'list'> 27


In [12]:
train['tokens'] = train["tokens"].apply(ast.literal_eval)
train['tags'] = train['tags'].apply(ast.literal_eval)

In [13]:
train.head()

,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"[مرحبا, Johan, Walker, ،, إشعار, الامتثال, :, ...","[O, B-FIRSTNAME, B-LASTNAME, O, O, O, O, O, O,...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"[نحن, نقوم, بمراجعة, القضية, التي, رفعها, كادي...","[O, O, O, O, O, O, B-FIRSTNAME, B-LASTNAME, O,...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"[بسبب, اتفاقية, السرية, ،, لا, يمكن, للمريضة, ...","[O, O, O, O, O, O, O, B-SEX, O, O, O, B-PREFIX...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"[عزيزي, ولي, الأمر, ،, يلزم, الحصول, على, إذن,...","[O, O, O, O, O, O, O, O, O, B-DATE, I-DATE, I-...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"[تنظيم, اجتماع, حول, 7, سنوات, (, Genderqueer,...","[O, O, O, B-AGE, I-AGE, O, B-GENDER, O, O, O, ...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [14]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


---

---



---



---



##### Use only the needed tags

In [ ]:
tags_to_mask =  [
 'EMAIL',
 'IBAN',
 'PERSONNAME',
 'IPV6',
 'MASKEDNUMBER',
 'CREDITCARDNUMBER',
 'LITECOINADDRESS',
 'BIC',
 'PHONENUMBER',
 'ORDINALDIRECTION',
 'VEHICLEVRM',
 'CREDITCARDINUMBER',
 'USERNAME',
 'IPV4',
 'PASSWORD',
 'MAC',
 'BITCOINADDRESS',
 'SECONDARYADDRESS',
 'CREDITCARDCVV',
 'PHONEIMEI',
 'ACCOUNTNAME',
 'BUILDINGNUMBER',
 'VEHICLEVIN',
 'STREET',
 'URL',
 'IP',
 'ZIPCODE',
 'ACCOUNTNUMBER',
 'FIRSTNAME',
 'LASTNAME',
 'MIDDLENAME',
 'NEARBYGPSCOORDINATE',
 'ETHEREUMADDRESS',
 'PIN',
 'SSN']

In [ ]:
def mask_tags(tags_list, tags_to_mask):
  """Masks specified tags in a list.

  Args:
    tags_list: A list of tags.
    tags_to_mask: A list of tags to be masked.

  Returns:
    A new list with the specified tags masked.
  """
  masked_tags = []
  for tag in tags_list:
    if tag[2:] in tags_to_mask:
      masked_tags.append("[MASKED]")
    else:
      masked_tags.append("O")
  return masked_tags

In [ ]:
train['masked_tags'] = train['tags'].apply(lambda x: mask_tags(x, tags_to_mask))

In [ ]:
train['masked_tags'][5:11]

,masked_tags
5,"[O, [MASKED], O, O, O, O, O, O, O, O, O, O, [M..."
6,"[O, O, O, O, O, O, O, O, O, O, O, O, [MASKED],..."
7,"[O, [MASKED], O, O, O, O, O, O, O, O, O, O, O,..."
8,"[O, O, O, O, O, O, O, O, [MASKED], [MASKED], O..."
9,"[O, O, O, O, O, O, O, O, O, [MASKED], O, O, O,..."
10,"[O, O, O, O, O, O, O, [MASKED], [MASKED], [MAS..."


In [ ]:
print(train['masked_tags'][0])
print(train['tokens'][0])
print(train['tags'][0])

['O', '[MASKED]', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', '[MASKED]', 'O', 'O', 'O', 'O', 'O']
['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'الامتثال', ':', 'تتطلب', 'التغييرات', 'الأخيرة', 'في', 'التشريعات', 'في', 'سانت', 'غالن', 'منك', 'تحديث', 'Suite', '559', 'الخاص', 'بك', 'في', 'سجلاتنا', '.']
['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STATE', 'I-STATE', 'O', 'O', 'B-SECONDARYADDRESS', 'I-SECONDARYADDRESS', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(train['masked_tags'][3])
print(train['tokens'][3])
print(train['tags'][3])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول', 'على', 'إذن', 'في', '2018', '-', '04', '-', '17T03', ':', '32', ':', '46', '.', '898Z', 'للرحلة', 'المدرسية', 'لـ', 'Maximillian', '.', 'يرجى', 'الرجوع', 'إلى', 'النموذج', 'المرفق', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.masked_tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


##### Creating new masked sentences

In [ ]:
import re

In [ ]:
def create_masked_sentence(row):
    """Creates a masked sentence from tokens and masked tags.

    Args:
    row: A pandas DataFrame row with 'tokens' and 'masked_tags' columns.

    Returns:
    A string representing the masked sentence.
    """
    masked_sentence_tokens = []
    for token, tag in zip(row['tokens'], row['masked_tags']):
        if tag == '[MASKED]':
            masked_sentence_tokens.append('[masked]')
        else:
            masked_sentence_tokens.append(token)

    return ' '.join(masked_sentence_tokens)

In [ ]:
# Apply the function to each row to create the new masked sentence column
train['masked_source'] = train.apply(create_masked_sentence, axis=1)

In [ ]:
print(train['masked_source'][3])
print(train['clean_source'][3])
print(train['masked_tags'][3])
print(train['tokens'][3])
print(train['tags'][3])

عزيزي ولي الأمر ، يلزم الحصول على إذن في 2018 - 04 - 17T03 : 32 : 46 . 898Z للرحلة المدرسية لـ [masked] . يرجى الرجوع إلى النموذج المرفق .
عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-04-17T03:32:46.898Z للرحلة المدرسية لـ Maximillian. يرجى الرجوع إلى النموذج المرفق.
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول', 'على', 'إذن', 'في', '2018', '-', '04', '-', '17T03', ':', '32', ':', '46', '.', '898Z', 'للرحلة', 'المدرسية', 'لـ', 'Maximillian', '.', 'يرجى', 'الرجوع', 'إلى', 'النموذج', 'المرفق', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
train.head()

,source,tokens,tags,clean_source,masked_tags,masked_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"[مرحبا, Johan, Walker, ،, إشعار, الامتثال, :, ...","[O, B-FIRSTNAME, B-LASTNAME, O, O, O, O, O, O,...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...,"[O, [MASKED], [MASKED], O, O, O, O, O, O, O, O...",مرحبا [masked] [masked] ، إشعار الامتثال : تتط...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"[نحن, نقوم, بمراجعة, القضية, التي, رفعها, كادي...","[O, O, O, O, O, O, B-FIRSTNAME, B-LASTNAME, O,...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...,"[O, O, O, O, O, O, [MASKED], [MASKED], O, O, O...",نحن نقوم بمراجعة القضية التي رفعها [masked] [m...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"[بسبب, اتفاقية, السرية, ،, لا, يمكن, للمريضة, ...","[O, O, O, O, O, O, O, B-SEX, O, O, O, B-PREFIX...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]",بسبب اتفاقية السرية ، لا يمكن للمريضة أنثى الد...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"[عزيزي, ولي, الأمر, ،, يلزم, الحصول, على, إذن,...","[O, O, O, O, O, O, O, O, O, B-DATE, I-DATE, I-...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",عزيزي ولي الأمر ، يلزم الحصول على إذن في 2018 ...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"[تنظيم, اجتماع, حول, 7, سنوات, (, Genderqueer,...","[O, O, O, B-AGE, I-AGE, O, B-GENDER, O, O, O, ...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...,"[O, O, O, O, O, O, O, O, O, O, O, O, [MASKED],...",تنظيم اجتماع حول 7 سنوات ( Genderqueer ) وسلام...


In [ ]:
# Save the train data after the updates
train.to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

# Loading Model

### SILMA-2B-4bit

In [ ]:
# !pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0-bnb-4bit" # Quantized version
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config,
)

In [ ]:
messages = [
    {"role": "user", "content": "كم عدد أيام الأسبوع؟"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

input_ids

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=256)

In [ ]:
tokenizer.decode(outputs[0]).split("<start_of_turn>")[-1]

### SILMA-2B

In [ ]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model="silma-ai/SILMA-Kashif-2B-Instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

In [ ]:
messages = [
    {"role": "user",
     "content": """ Mask the PII as [masked] in the given source sentence

     source: I sent my email address M.helmy@nu.edu.eg to my supervisor dr. gahda khoriba in Nile University.

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
response

In [ ]:
messages = [
    {"role": "user",
     "content": f""" Mask the PII as [masked] in the given source sentence

     source: {df["source"][0]}

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
print(df["source"][0])
print(response)